In [1]:
# Load all relevant modules
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import time as timer
import math

In [3]:
def create_batch(x, y, batch_size=32):
    n  = x.shape[0] # number of data
    batches = [] 
    batch_num = math.floor(n / batch_size)
    print(f"batch num: {batch_num}")

    for i in range(batch_num):
        x_batch = x[i * batch_size:(i+1) * batch_size, :]
        y_batch = y[i * batch_size:(i+1) * batch_size]
        batches.append((x_batch, y_batch))

    if n % batch_size != 0:
        x_batch = x[batch_num * batch_size:, :]
        y_batch = y[batch_num * batch_size:]
        batches.append((x_batch, y_batch))
    
    return batches

In [2]:
class LogisticClassifier:
    def __init__(self, step_size=1e-7):
        self.step_size = step_size
        self.w = None
        self.eps = 1e-20
    
    def initialize_weights(self, d=1):
        self.w = np.random.normal(0, 0.01, (d+1, ))

    def sigmoid(self, z):
        return 1 / (1 + np.exp((-z + self.eps)) + self.eps)

    def model_fn(self, X):
        return self.sigmoid(X @ self.w)
    
    def loss_fn(self, y, prob_hat, mode='mean'):
        if mode == 'sum':
            loss = -np.sum((y * np.log(prob_hat + self.eps) + (1 - y) * np.log(1 - prob_hat + self.eps)))
        else:
            loss = -np.mean((y * np.log(prob_hat + self.eps) + (1 - y) * np.log(1 - prob_hat + self.eps)))

        return loss
    
    def grad_loss_fn(self, X, prob_hat, y):
        n = X.shape[0]
        g = ((prob_hat - y) @ X) / n
        return g

    def accuracy_fn(self, y, prob_hat):
        y_hat = np.round(prob_hat)
        acc = np.abs(y - y_hat) < self.eps
        return np.sum(acc) / acc.shape[0]

    def train_on_step(self, X, y):
        """
        Args:
            X (nd.array): n x d
            y
        """
        Xb = np.insert(X, 0, 1, axis=1) # add 1s for bias term
        yb = y

        # Forward pass
        probb_hat = self.model_fn(Xb, self.w)
        
        loss_val = self.loss_fn(yb, probb_hat)
        
        # Backward pass: gradient update
        g = self.grad_loss_fn(Xb, yb, probb_hat)
        self.w = self.w - self.step_size * g

        return loss_val, probb_hat
    

    def fit(self, X, y, epochs=1, batch_size=32):
        
        batches = create_batch(X, y, batch_size=batch_size)
        num_batches = len(batches)

        d = X.shape[1]
        self.initialize_weights(d=d)
        
        losses = []
        for ep in range(epochs):
            loss_avg = 0.
            start_t = timer.time()
            for b, (X_batch, y_batch) in enumerate(batches):
                # Train on step or batch
                l, prob_hat = self.train_on_step(X_batch, y_batch)

                # Compute accuracy
                acc = self.accuracy_fn(y_batch, prob_hat)

                loss_avg += l

            loss_avg /= num_batches
            losses.append(loss_avg)
            

                


